# 4. nested config

I'm really tired of two things:
 * rummaging around in complex unstructured config data
 * parsing unstructured config data by hand

Fortunately, there's a better way!

Here's the actual config[<sup>1</sup>](#fn1) classes from my current work project. (Annoyingly, the combination of type annotations and factory parameters forces a bottom-up ordering even though a top-down ordering would make more sense.)

<span id="fn1">[1] They also do the actual work of fetching and processing things, but that's an implementation detail.</span>

----

We start with some imports. Since we've elided the bulk of the implementation code, all we need are some typing helpers and `attr`.

In [ ]:
from typing import Dict, List, Optional, Union

import attr

Base class for `Source` objects. This isn't instantiated directly, but all sources subclass it.

There's some trickery here regarding the `type` field. We put that in the YAML to indicate which source type we want, but we don't actually pass it into the constructor. However, we need it again when we serialize a config. (We can't use a non-attr `type` field for that because `attrs` won't serialize it.)

There's some more trickery regarding `logger`. A naïve implementation using `self._logger = logger` is foiled by `frozen=True`, so we have to sneak up from behind by calling `object.__setattr__` directly. Nothing is ever truly immutable in Python.

In [ ]:
@attr.s(frozen=True)
class SourceBase:
    "Base class: ad-hoc interface and some common behaviour."

    name: str = attr.ib()
    # We strip the type field out of the source dict during parsing. This
    # attrib puts it back during serialization so we can read what we wrote.
    type: str = attr.ib(init=False)

    @type.default
    def _type_default(self):
        return self._source_type

    _logger = None

    @property
    def displayname(self):
        return f"{self._source_type}:{self.name}"

    @property
    def logger(self):
        if self._logger is None:
            # Because we (and all our subclasses) are frozen, we need to cheat.
            logger = _sources_logger.child(self.displayname)
            object.__setattr__(self, "_logger", logger)
        return self._logger

    async def fetch(self, dest_dir):
        raise NotImplementedError()

The simplest source type.

In [ ]:
@attr.s(frozen=True)
class Manifest(SourceBase):
    "Fetch a file that is assumed to contain YAML."

    _source_type = "manifest"

    url: str = attr.ib()

    async def fetch(self, dest_dir):
        pass

A `HelmChart` has two slightly complicated fields:
 * `templatevars` is a dict (which is passed into the constructor as-is from the YAML)
 * `releasevars` is another `attr.s` class that must be constructed before we pass it in
 
Warning: While `HelmChart` is frozen and thus "immutable", `HelmChart.templatevars` is a normal mutable dict. We could add `frozendict` as a dependency if we really cared, but that comes with its own caveats. Nothing is ever truly immutable in Python.

In [ ]:
@attr.s(frozen=True)
class HelmChartReleaseVars:
    "Things helm wnats that aren't template variables."

    name: Optional[str] = attr.ib(default=None)
    namespace: Optional[str] = attr.ib(default=None)


@attr.s(frozen=True)
class HelmChart(SourceBase):
    "Fetch a helm chart and render it to a single YAML file."

    _source_type = "chart"

    repo: str = attr.ib()
    version: str = attr.ib()
    templatevars: Dict[str, str] = attr.ib(factory=dict)
    releasevars: HelmChartReleaseVars = attr.ib(factory=HelmChartReleaseVars)

    async def fetch(self, dest_dir):
        pass

`Archive` has the most complicated field of all: `paths` is a list of `ArchivePath` objects, which must all be constructed on the way in. `ArchivePath` also does some custom validation on one of its fields.

(As before, `frozen=True` can't save us from `list`'s mutability. We could use a `tuple` instead, but then we'd have to do more type juggling between these classes and YAML.)

In [ ]:
@attr.s(frozen=True)
class ArchivePath:
    "Complicated path glob matcher."

    path: str = attr.ib()
    dest: str = attr.ib(default="")

    @path.validator
    def _path_validator(self, attribute, value):
        for seg in value.split("/")[:-1]:
            if seg == "**":
                raise ValueError("** may only appear at the end of a path")

    def match(self, path) -> Optional[str]:
        pass


@attr.s(frozen=True)
class Archive(SourceBase):
    "Fetch an archive and extracts a subset of the files within it."

    _source_type = "archive"

    url: str = attr.ib()
    paths: List[ArchivePath] = attr.ib(factory=list)

    async def fetch(self, dest_dir):
        pass

`SOURCE_MAP` is how we get from the `type` field in the YAML to the source class and `Source = Union[...]` tells anything looking at type annotations that a `Source` is an instance of one of our three source classes.

In [ ]:
SOURCE_MAP = {"chart": HelmChart, "manifest": Manifest, "archive": Archive}

Source = Union[HelmChart, Manifest, Archive]

Finally, the top-level config class. When I'm finished, there will be more than just the `sources` field in here.

In [ ]:
@attr.s(frozen=True)
class Config:
    "Structured config object parsed populated from YAML file."

    sources: List[Source] = attr.ib()